# Modeling

- TODO: add notebook description

## Table of contents:

* [1. Data loading and preprocessing](#first-enumeration)

In [40]:
import os

import numpy as np
import pandas as pd
from datetime import datetime
import time

import xgboost as xgb

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

import holidays

In [41]:
# Global variable for data relative path
DATA_PATH = os.path.abspath("../data/inputs")
RESULTS_PATH = os.path.abspath("../data/outputs")
TRAINED_MODELS_PATH = os.path.abspath("../trained_models")

## 1. Data loading and preprocessing

### 1.1 Data loading

In [3]:
data_train = pd.read_csv(os.path.join(DATA_PATH, "train.csv"))
data_test = pd.read_csv(os.path.join(DATA_PATH, "test.csv"))

### 1.2 Data preprocessing

In [4]:
def is_holiday_week(data):

    #get holiday dates in France from 2012 to 2017
    holidays_france = pd.DataFrame(
        holidays.France(years=range(2012, 2018)).keys(),
        dtype="datetime64[ns]",
        columns=["holiday_date"])

    # make a tuple of (year, week of year)
    holidays_france["year"] = holidays_france["holiday_date"].dt.year
    holidays_france["week"] = holidays_france["holiday_date"].dt.isocalendar().week

    year_week_tuple = list(holidays_france[["year", "week"]].itertuples(index=False, name=None))

    # check each row in the data if it belongs to (year, week of the year) tuple
    return pd.Series(list(zip(data.year, data.week)), index=data.index).isin(year_week_tuple)

In [5]:
def process_dates(data):

    # dates preprocessing
    data["year"] =  data.day_id.dt.year
    data["month"] = data.day_id.dt.month
    data["week"] = data.day_id.dt.isocalendar().week
    data["quarter"] = data.day_id.dt.quarter

    # define the 4 seasons of the year based on months
    #seasons = [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1]
    #month_to_season = dict(zip(range(1,13), seasons))
    #data["season"] = data.index.month.map(month_to_season)

    #either a day in the weekly turnover belongs to a holiday
    data["is_holiday"] = is_holiday_week(data).astype(int)

    return data

In [6]:
def one_hot_encoding(data, categorical_columns, training=True):
    """add one hot encoding of categorical columns"""
    
    if training:
        global ohe # not recommended doing so, but it's the simplest solution
        ohe = OneHotEncoder()
        one_hot_encoded_data = ohe.fit_transform(data[categorical_columns])
        
    else:
        one_hot_encoded_data = ohe.transform(data[categorical_columns])
        
    
    one_hot_df = pd.DataFrame(one_hot_encoded_data.toarray(),
                          columns=ohe.get_feature_names_out(),
                          index=data.index)
        
    return one_hot_df

In [7]:
def add_turnover_lags(data, time_lag=4):
    """add historical data of the last time_lag year"""
    
    # add empty columns to fill lags
    for i in range(time_lag):
        lag = np.empty(data.shape[0])
        lag[:] = np.nan
        data["turnover_N-{}".format(i+1)] = lag
    
    # get the list of departments and stores
    business_units_list = data.but_num_business_unit.unique()
    department_list = data.dpt_num_department.unique()
    
    # ingest lags by store and by department
    for i in business_units_list:
        for j in department_list:
            for k in range(1, time_lag+1):
                lag_data = data.loc[(data.but_num_business_unit==i) & (data.dpt_num_department==j), "turnover"].shift(-52*k)
                if lag_data.shape !=0 :
                    data.loc[lag_data.index, "turnover_N-{}".format(k)]= lag_data
    
    return data

In [8]:
def process_data(data, categorical_columns, training=True, time_lag=4):
    
    # set day_id adequate type
    data["day_id"] = pd.to_datetime(data["day_id"], infer_datetime_format=True)
    
    # sort data by day_id 
    data.sort_values("day_id", ascending=False, inplace=True)
    
    # process dates
    _ = process_dates(data)
    
    # add time lags 
    add_turnover_lags(data, time_lag=time_lag)
    
    # one hot encoding
    one_hot_encoded_data = one_hot_encoding(data, categorical_columns, training)
    
    # drop old categorical columns
    data.drop(columns=categorical_columns, inplace=True)
    
    #concat with the one hot encoded ones
    data = pd.concat([data, one_hot_encoded_data], axis=1)

    return data
    

In [9]:
data_train = process_data(data_train,
                          categorical_columns= ["dpt_num_department", "but_num_business_unit", "year", "month", "week", "quarter"],
                          training=True)

### 1.3 Split train/eval sets


In [10]:
# train on all data except the last month
train_idx = data_train.day_id.dt.date <= datetime(year=2017, month=8, day=31).date()

In [11]:
X = data_train.drop(labels=['turnover', "day_id"], axis=1)
y = data_train['turnover']

In [12]:
X_train, y_train = X.loc[train_idx], y.loc[train_idx]
X_eval, y_eval = X.loc[~train_idx], y.loc[~train_idx]

In [13]:
print("X_train shape: {}, y_train shape: {}".format(X_train.shape, y_train.shape))
print("X_eval shape: {}, y_eval shape: {}".format(X_eval.shape, y_eval.shape))

X_train shape: (271369, 406), y_train shape: (271369,)
X_eval shape: (6350, 406), y_eval shape: (6350,)


## 2. Train an XGboost regressor

Since Xgboost deals with missing values, we will not remove these

In [14]:
xgb_reg = xgb.XGBRegressor(n_estimators=1, n_jobs=-1, max_depth= 20, verbosity=1, random_state=42)

In [15]:
xgb_reg.fit(X_train.astype(float), y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=20, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1, n_jobs=-1,
             num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [43]:
# save the model 
xgb_reg.save_model(os.path.join(TRAINED_MODELS_PATH, "XGBoost_{}".format(time.strftime("%Y%m%d-%H%M%S"))))

## 3. Evaluate the model

We will use for evaluation MAE and MSE

In [16]:
# get the predictions
y_pred = xgb_reg.predict(X_eval.astype(float))

In [17]:
print("MAE for eval data: {}".format(mean_absolute_error(y_eval, y_pred)))
print("MSE for eval data: {}".format(mean_squared_error(y_eval, y_pred)))

MAE for eval data: 246.62957350810123
MSE for eval data: 196182.54886070776


## 4. Make predictions on the test set

In [18]:
def preprocess_test_data(test_data, historical_data_path):
    """get historical data from data_train"""
    
    # add an empty turnover column
    empty_column = np.empty(test_data.shape[0])
    empty_column[:] = np.nan
    test_data["turnover"] = empty_column
    
    historical_data = pd.read_csv(historical_data_path)
    
    return pd.concat([historical_data, test_data], axis=0, ignore_index=True)

In [19]:
# add lags to data_test from historical data
data_test = preprocess_test_data(data_test, os.path.join(DATA_PATH, "train.csv"))

In [20]:
data_test_copy = data_test[data_test["turnover"].isna()].copy()

In [21]:
#process data_test
data_test = process_data(data_test, 
                    categorical_columns= ["dpt_num_department", "but_num_business_unit", "year", "month", "week", "quarter"], 
                    training=False)

In [22]:
#drop train data
data_test = data_test[data_test["turnover"].isna()]

In [23]:
#Get X_test data
X_test = data_test.drop(columns=["turnover", "day_id"])

In [24]:
print("X_test shape: {}".format(X_test.shape))

X_test shape: (10136, 406)


In [35]:
#make predictions
y_pred = xgb_reg.predict(X_test.astype(float))

# concat y_pred with X_test
y_pred = pd.DataFrame(y_pred, columns=["turnover_pred"], index=data_test.index)
results = pd.concat([data_test_copy, y_pred], axis=1)
results.head()

,day_id,but_num_business_unit,dpt_num_department,turnover,turnover_pred
277719,2017-11-25,95,73,NaN,7.378144
277720,2017-11-25,4,117,NaN,355.700226
277721,2017-11-25,113,127,NaN,222.167099
277722,2017-11-25,93,117,NaN,154.831985
277723,2017-11-25,66,127,NaN,637.350830


Save results

In [38]:
results.to_csv(os.path.join(RESULTS_PATH, "predictions.csv"))